In [ ]:
import time
import random
import sklearn
import sklearn.datasets
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cvxpy import *
from scipy import stats
from scipy import sparse
from scipy import linalg
from scipy.linalg import eigh
from scipy.sparse import linalg
from scipy.sparse import csr_array
from scipy.sparse import coo_array
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import eigs, eigsh
from sklearn.preprocessing import normalize
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.pairwise import euclidean_distances

In [ ]:
def sigma_mat(dist, k):
  dist.sort(axis=1)
  dist_vec = np.array([dist[:, k-1]])
  sigma_mat = 2 * np.outer(dist_vec, dist_vec)
  return sigma_mat


def DM_coordinates(v1, v2, k):
  distsI = euclidean_distances(v1, v1)
  dist_matI = np.copy(distsI)
  sigma_matI = sigma_mat(dist_matI, k)

  KI = np.exp(-distsI**2 / sigma_matI)
  KI = KI - np.diag(np.diag(KI))
  DI = np.diag(np.sum(KI, axis=1))
  D_leftI = np.diag(np.sum(KI,axis = 1)**-0.5)
  D_rightI = np.diag(np.sum(KI,axis = 1)**0.5)
  rand_walk_LI = np.matmul(np.linalg.inv(DI), KI)
  RWL_primeI = np.matmul(D_rightI, np.matmul(rand_walk_LI, D_leftI))

  eigenValuesI, eigenVectorsI = eigh(RWL_primeI)
  idxI = eigenValuesI.argsort()[::-1]
  d_lamI = np.diag(eigenValuesI[idxI])
  eig_I = D_leftI@eigenVectorsI[:, idxI]

  diffusion_coordinatesI = np.matmul(eig_I, d_lamI)
  diffusion_coordinatesI_df = pd.DataFrame(diffusion_coordinatesI)

  distsII = euclidean_distances(v2, v2)
  dist_matII = np.copy(distsII)
  sigma_matII = sigma_mat(dist_matII, k)

  KII = np.exp(-distsII**2 / sigma_matII)
  KII = KII - np.diag(np.diag(KII))
  DII = np.diag(np.sum(KII, axis=1))
  D_leftII = np.diag(np.sum(KII,axis = 1)**-0.5)
  D_rightII = np.diag(np.sum(KII,axis = 1)**0.5)
  rand_walk_LII = np.matmul(np.linalg.inv(DII), KII)
  RWL_primeII = np.matmul(D_rightII, np.matmul(rand_walk_LII, D_leftII))

  eigenValuesII, eigenVectorsII = eigh(RWL_primeII)
  idxII = eigenValuesII.argsort()[::-1]
  d_lamII = np.diag(eigenValuesII[idxII])
  eig_II = D_leftII@eigenVectorsII[:, idxII]

  diffusion_coordinatesII = np.matmul(eig_II, d_lamII)
  diffusion_coordinatesII_df = pd.DataFrame(diffusion_coordinatesII)
  return diffusion_coordinatesI_df, diffusion_coordinatesII_df


def check_conflicts(n, eigenvec):
  if eigenvec.shape[1] == 1:
    x_star = eigenvec
  else:
    d = eigenvec.shape[1]
    x_star = eigenvec[:, (d - 1)]

  list_x = range(1, n+1)
  list_y = range(1, n+1)

  pairs = []
  for x in list_x:
    for y in list_y:
      pairs.append((x, y))

  df = pd.DataFrame(pairs)
  df["x_star"] = x_star
  df.columns = ["row_idx", "col_idx", "x_star"]
  df_new = df

  first_point = []
  second_point = []

  for i in range(n):
    idx = np.argmax(df_new.x_star)
    r_idx = df_new.iloc[idx, 0]
    first_point.append(r_idx)
    c_idx = df_new.iloc[idx, 1]
    second_point.append(c_idx)
    df_new = df_new[df_new.row_idx != r_idx]
    df_new = df_new[df_new.col_idx != c_idx]

  one_to_one = pd.DataFrame(first_point)
  one_to_one["second_point"] = second_point
  one_to_one.columns = ["first_point", "second_point"]
  return one_to_one


def spearman_rate(pairs, v1, v2):
  idx1 = pairs.iloc[:,0]
  idx1 = idx1 - 1
  vec1 = v1.iloc[idx1,0]

  idx2 = pairs.iloc[:,1]
  idx2 = idx2 - 1
  vec2 = v2.iloc[idx2,0]

  correlation_coefficient, p_value = stats.spearmanr(vec1, vec2)
  return correlation_coefficient


def data_generator_path_graph(n):
  view_1 = {'x_coord': np.sort(np.random.uniform(0, 100, n)), 'y_coord': np.repeat(1, n), 'z_coord': np.repeat(0, n)}
  view_1 = pd.DataFrame(view_1)
  view_2 = {'x_coord': np.sort(np.random.uniform(0, 100, n)), 'y_coord': np.repeat(-1, n), 'z_coord': np.repeat(0, n)}
  view_2 = pd.DataFrame(view_2)
  return view_1, view_2


def kronecker_product2(A, B, n):
  a_w = np.where(A != 0)
  b_w = np.where(B != 0)
  r_a = list(a_w[0])
  c_a = list(a_w[1])
  r_b = list(b_w[0])
  c_b = list(b_w[1])

  R1 = (np.outer(r_a, np.ones((1, len(r_b)))) * n).flatten() + (np.outer(np.ones((len(r_a), 1)), r_b)).flatten()
  C1 = (np.outer(c_a, np.ones((1, len(c_b)))) * n).flatten() + (np.outer(np.ones((len(c_a), 1)), c_b)).flatten()
  V1 = (np.outer(A[r_a, c_a].flatten(), np.ones((1, len(r_b))))) * (np.outer(np.ones((len(c_a), 1)), B[r_b, c_b].flatten()))
  res = coo_array((V1.flatten(), (R1, C1)), shape = (n*n, n*n))
  return res


def compute_M2(X, Y, k):
  x_dist = euclidean_distances(X,X)
  idx_x = np.argpartition(x_dist, range(k))[:, k:]
  x_dist[np.arange(idx_x.shape[0])[:,None], idx_x] = 2
  x_dist = 2 - x_dist
  x_dist = 0.5*(x_dist+x_dist.T)

  y_dist = euclidean_distances(Y,Y)
  idx_y = np.argpartition(y_dist, range(k))[:, k:]
  y_dist[np.arange(idx_y.shape[0])[:,None], idx_y] = 2
  y_dist = 2 - y_dist
  y_dist = 0.5*(y_dist+y_dist.T)

  n = x_dist.shape[0]
  M2 = kronecker_product2(x_dist, y_dist, n)
  return M2


def kronecker_product3(A, B, n, sig_d = 5):
  a_w = np.where(A != 0)
  b_w = np.where(B != 0)
  r_a = list(a_w[0])
  c_a = list(a_w[1])
  r_b = list(b_w[0])
  c_b = list(b_w[1])

  R1 = (np.outer(r_a, np.ones((1, len(r_b)))) * n).flatten() + (np.outer(np.ones((len(r_a), 1)), r_b)).flatten()
  C1 = (np.outer(c_a, np.ones((1, len(c_b)))) * n).flatten() + (np.outer(np.ones((len(c_a), 1)), c_b)).flatten()
  V1 = (np.outer(A[r_a, c_a].flatten(), np.ones((1, len(r_b))))) - (np.outer(np.ones((len(c_a), 1)), B[r_b, c_b].flatten()))
  V = np.where(np.abs(V1) >= 3*sig_d, 0, (4.5 - ((V1**2)/(2*(sig_d**2)))))
  res = coo_array((V.flatten(), (R1, C1)), shape = (n*n, n*n))
  return res


def compute_M3(X, Y):
  x_dist = euclidean_distances(X,X)
  y_dist = euclidean_distances(Y,Y)

  n = x_dist.shape[0]
  M3 = kronecker_product3(x_dist, y_dist, n)
  return M3


def kronecker_product4(A, B, n):
  a_w = np.where(A != 0)
  b_w = np.where(B != 0)
  r_a = list(a_w[0])
  c_a = list(a_w[1])
  r_b = list(b_w[0])
  c_b = list(b_w[1])

  R1 = (np.outer(r_a, np.ones((1, len(r_b)))) * n).flatten() + (np.outer(np.ones((len(r_a), 1)), r_b)).flatten()
  C1 = (np.outer(c_a, np.ones((1, len(c_b)))) * n).flatten() + (np.outer(np.ones((len(c_a), 1)), c_b)).flatten()
  V1 = (np.outer(A[r_a, c_a].flatten(), np.ones((1, len(r_b))))) - (np.outer(np.ones((len(c_a), 1)), B[r_b, c_b].flatten()))
  V = np.exp(-1*(V1**2))
  res = coo_array((V.flatten(), (R1, C1)), shape = (n*n, n*n))
  return res


def compute_M4(X, Y):
  x_dist = euclidean_distances(X,X)
  y_dist = euclidean_distances(Y,Y)

  n = x_dist.shape[0]
  M4 = kronecker_product4(x_dist, y_dist, n)
  return M4


def get_constraint_matrix(n):
    upper_blocks = np.zeros((n, n**2))
    for i in range(n):
      upper_blocks[i, i*n:((i*n)+n)] = 1

    lower_blocks = np.zeros((n, n**2))
    for j in range(n):
      lower_blocks[:,j*n:((j*n)+n)] = np.eye(n)

    A = np.vstack((upper_blocks, lower_blocks))
    return A[:-1,:]


def SMAC(C, b, W):
    k = C.shape[0]
    Ik = np.zeros((k-1, k))
    Ik[:, :-1] = np.eye(k-1)
    for i in range(k):
        C[i] = C[i] - b[i]/b[-1]*C[-1]
    Ceq = np.dot(Ik, C)
    inv_C = np.linalg.inv(np.dot(Ceq, Ceq.T))
    all_C = np.dot(Ceq.T, np.dot(inv_C, Ceq))
    Pc = np.eye(C.shape[1])-all_C
    new_W = np.dot(Pc, np.dot(W, Pc))
    eValue, eVector = eigsh(new_W, k=1)
    return eVector


def SDP_relax(W, n):
    """
    :param W: Compatibility matrix
    :param n: size the graphs
    :return: optimal x
    """
    d = np.diag(W)
    Weq = np.zeros((W.shape[0]+1, W.shape[1]+1))
    Weq[0, 1:] = d/2
    Weq[1:, 0] = d/2
    Weq[1:, 1:] = W-np.diag(d)

    c, A = [], []

    # First condition: X[0, 0] = 1
    A_11 = np.zeros(Weq.shape)
    A_11[0, 0] = 1
    c.append(1), A.append(A_11)

    # Second condition: weak enforcement of x_i = x_i^2
    for i in range(1, W.shape[0]+1):
        Ai = np.zeros(Weq.shape)
        Ai[0, i] = -1
        Ai[i, 0] = -1
        Ai[i, i] = 2
        c.append(0), A.append(Ai)

    # Conditions of Sum_{j}(x_i*n+j) for all i, same for columns
    for s in range(n):
        As = np.zeros(Weq.shape)
        Asp = np.zeros(Weq.shape)
        As[n*s+1:n*(s+1)+1, n*s+1:n*(s+1)+1] = np.eye(n)
        for t in range(n):
            Asp[n*t+s+1, n*t+s+1] = 1
        c.append(1), A.append(As)
        c.append(1), A.append(Asp)

    X = Variable(shape=(n**2+1, n**2+1), PSD=True)
    obj = Maximize(trace(Weq*X))
    constraints = []
    for ci, Ai in zip(c, A):
        constraints.append(trace(matmul(Ai, X))==ci)
    prob = Problem(obj, constraints)
    prob.solve(SCS)
    if prob.status != 'optimal':
        print('CVXPY failed to reach optimal value')
    return np.diag(X.value)[1:]

In [ ]:
def compute_M2_new(X, Y, k):
  x_dist = euclidean_distances(X,X)
  idx_x = np.argpartition(x_dist, range(k))[:, k:]
  x_dist[np.arange(idx_x.shape[0])[:,None], idx_x] = 2
  x_dist = 2 - x_dist
  Q_x = 0.5*(x_dist+x_dist.T)

  y_dist = euclidean_distances(Y,Y)
  idx_y = np.argpartition(y_dist, range(k))[:, k:]
  y_dist[np.arange(idx_y.shape[0])[:,None], idx_y] = 2
  y_dist = 2 - y_dist
  Q_y = 0.5*(y_dist+y_dist.T)

  return Q_x, Q_y

In [ ]:
def matching_rate_M2_mixed_CC(n, k, n_col, num_coords, M, data_func, *args):
  st = time.time()

  num_ev = []
  cor_lst = []
  cor_lst1 = []
  cor_lst2 = []
  cor_lst3 = []
  cor_lst4 = []
  cor_lst5 = []
  cor_lst6 = []
  cor_lst7 = []
  cor_lst8 = []
  cor_lst9 = []

  for i in range(M):
    print(i)
    view_1, view_2 = data_func(*args)
    view1 = view_1.iloc[:,0:n_col]
    view2 = view_2.iloc[:,0:n_col]

    diffusion_coordinatesI_df, diffusion_coordinatesII_df = DM_coordinates(view1, view2, k)

    leadEV1 = diffusion_coordinatesI_df.iloc[:,1:num_coords]
    leadEV2 = diffusion_coordinatesII_df.iloc[:,1:num_coords]

    norm_view1 = normalize(leadEV1, axis=1, norm='l2')
    norm_view2 = normalize(leadEV2, axis=1, norm='l2')

    Q_1, Q_2 = compute_M2_new(norm_view1, norm_view2, k)
    e_val_Q1, e_vec_Q1 = eigsh(Q_1, k=3)
    e_val_Q2, e_vec_Q2 = eigsh(Q_2, k=3)

    V = []
    for i in range(3):
      for j in range(3):
        V.append(np.reshape(np.outer(e_vec_Q1[:,i],e_vec_Q2[:,j]),(n**2)))

    eVector1 = V[8]
    eVector2 = V[7]
    eVector3 = V[6]
    eVector4 = V[5]
    eVector5 = V[4]
    eVector6 = V[3]
    eVector7 = V[2]
    eVector8 = V[1]
    eVector9 = V[0]

    pairs1 = check_conflicts(n, eVector1.reshape(n*n, 1))
    pairs2 = check_conflicts(n, eVector2.reshape(n*n, 1))
    pairs3 = check_conflicts(n, eVector3.reshape(n*n, 1))
    pairs4 = check_conflicts(n, eVector4.reshape(n*n, 1))
    pairs5 = check_conflicts(n, eVector5.reshape(n*n, 1))
    pairs6 = check_conflicts(n, eVector6.reshape(n*n, 1))
    pairs7 = check_conflicts(n, eVector7.reshape(n*n, 1))
    pairs8 = check_conflicts(n, eVector8.reshape(n*n, 1))
    pairs9 = check_conflicts(n, eVector9.reshape(n*n, 1))

    s1 = spearman_rate(pairs1, view_1, view_2)
    cor_lst1.append(s1)
    s2 = spearman_rate(pairs2, view_1, view_2)
    cor_lst2.append(s2)
    s3 = spearman_rate(pairs3, view_1, view_2)
    cor_lst3.append(s3)
    s4 = spearman_rate(pairs4, view_1, view_2)
    cor_lst4.append(s4)
    s5 = spearman_rate(pairs5, view_1, view_2)
    cor_lst5.append(s5)
    s6 = spearman_rate(pairs6, view_1, view_2)
    cor_lst6.append(s6)
    s7 = spearman_rate(pairs7, view_1, view_2)
    cor_lst7.append(s7)
    s8 = spearman_rate(pairs8, view_1, view_2)
    cor_lst8.append(s8)
    s9 = spearman_rate(pairs9, view_1, view_2)
    cor_lst9.append(s9)

    v1 = np.zeros((n*n,1))
    n_pairs1 = pairs1.sort_values('first_point')
    for i in range(n):
      d1 = n_pairs1.iloc[i,1]
      idx1 = (n*i) + d1-1
      v1[idx1] = 1
    P_v1 = v1.reshape(n, n)
    result_matrix1 = Q_2 @ P_v1 @ Q_1.T @ P_v1.T
    obj_res1 = np.trace(result_matrix1)

    v2 = np.zeros((n*n,1))
    n_pairs2 = pairs2.sort_values('first_point')
    for i in range(n):
      d2 = n_pairs2.iloc[i,1]
      idx2 = (n*i) + d2-1
      v2[idx2] = 1
    P_v2 = v2.reshape(n, n)
    result_matrix2 = Q_2 @ P_v2 @ Q_1.T @ P_v2.T
    obj_res2 = np.trace(result_matrix2)

    v3 = np.zeros((n*n,1))
    n_pairs3 = pairs3.sort_values('first_point')
    for i in range(n):
      d3 = n_pairs3.iloc[i,1]
      idx3 = (n*i) + d3-1
      v3[idx3] = 1
    P_v3 = v3.reshape(n, n)
    result_matrix3 = Q_2 @ P_v3 @ Q_1.T @ P_v3.T
    obj_res3 = np.trace(result_matrix3)

    v4 = np.zeros((n*n,1))
    n_pairs4 = pairs4.sort_values('first_point')
    for i in range(n):
      d4 = n_pairs4.iloc[i,1]
      idx4 = (n*i) + d4-1
      v4[idx4] = 1
    P_v4 = v4.reshape(n, n)
    result_matrix4 = Q_2 @ P_v4 @ Q_1.T @ P_v4.T
    obj_res4 = np.trace(result_matrix4)

    v5 = np.zeros((n*n,1))
    n_pairs5 = pairs5.sort_values('first_point')
    for i in range(n):
      d5 = n_pairs5.iloc[i,1]
      idx5 = (n*i) + d5-1
      v5[idx5] = 1
    P_v5 = v5.reshape(n, n)
    result_matrix5 = Q_2 @ P_v5 @ Q_1.T @ P_v5.T
    obj_res5 = np.trace(result_matrix5)

    v6 = np.zeros((n*n,1))
    n_pairs6 = pairs6.sort_values('first_point')
    for i in range(n):
      d6 = n_pairs6.iloc[i,1]
      idx6 = (n*i) + d6-1
      v6[idx6] = 1
    P_v6 = v6.reshape(n, n)
    result_matrix6 = Q_2 @ P_v6 @ Q_1.T @ P_v6.T
    obj_res6 = np.trace(result_matrix6)

    v7 = np.zeros((n*n,1))
    n_pairs7 = pairs7.sort_values('first_point')
    for i in range(n):
      d7 = n_pairs7.iloc[i,1]
      idx7 = (n*i) + d7-1
      v7[idx7] = 1
    P_v7 = v7.reshape(n, n)
    result_matrix7 = Q_2 @ P_v7 @ Q_1.T @ P_v7.T
    obj_res7 = np.trace(result_matrix7)

    v8 = np.zeros((n*n,1))
    n_pairs8 = pairs8.sort_values('first_point')
    for i in range(n):
      d8 = n_pairs8.iloc[i,1]
      idx8 = (n*i) + d8-1
      v8[idx8] = 1
    P_v8 = v8.reshape(n, n)
    result_matrix8 = Q_2 @ P_v8 @ Q_1.T @ P_v8.T
    obj_res8 = np.trace(result_matrix8)

    v9 = np.zeros((n*n,1))
    n_pairs9 = pairs9.sort_values('first_point')
    for i in range(n):
      d9 = n_pairs9.iloc[i,1]
      idx9 = (n*i) + d9-1
      v9[idx9] = 1
    P_v9 = v9.reshape(n, n)
    result_matrix9 = Q_2 @ P_v9 @ Q_1.T @ P_v9.T
    obj_res9 = np.trace(result_matrix9)

    obj_arr = np.array([float(obj_res1), float(obj_res2), float(obj_res3), float(obj_res4), float(obj_res5), float(obj_res6), float(obj_res7), float(obj_res8), float(obj_res9)])
    idx = np.argmax(obj_arr)
    max = np.max(obj_arr)
    print("The result - ev number " + str((idx+1)) + ", objective function = " + str(round(max, 3)))

    s_arr = np.array([float(s1), float(s2), float(s3), float(s4), float(s5), float(s6), float(s7), float(s8), float(s9)])
    cor_lst.append(s_arr[idx])
    num_ev.append(idx+1)

  et = time.time()

  total_time = et - st
  print("Execution time of matching_rate: ", total_time, " seconds")

  return pairs1, pairs2, pairs3, pairs4, pairs5, pairs6, pairs7, pairs8, pairs9, view_1, view_2, cor_lst1, cor_lst2, cor_lst3, cor_lst4, cor_lst5, cor_lst6, cor_lst7, cor_lst8, cor_lst9, num_ev, cor_lst

In [ ]:
n_lst = [50, 60, 70, 80, 90, 100, 120, 150]

l = len(n_lst)
num_coords = 4
n_col = 1

st = time.time()

def GM_sys_M2_mixed_CC(ns, M, num_coords, n_col):
  num_nodes = []
  run_time = []
  num_evs = []

  median_spe_M = []
  median_spe_M_1 = []
  median_spe_M_2 = []
  median_spe_M_3 = []

  mean_spe_M = []
  mean_spe_M_1 = []
  mean_spe_M_2 = []
  mean_spe_M_3 = []

  std_spe_M = []
  std_spe_M_1 = []
  std_spe_M_2 = []
  std_spe_M_3 = []

  for i in range(l):
    print(i)
    num_nodes.append(ns[i])
    st = time.time()
    k_neigh = 15

    pairs1, pairs2, pairs3, pairs4, pairs5, pairs6, pairs7, pairs8, pairs9, view_1, view_2, cor_lst1, cor_lst2, cor_lst3, cor_lst4, cor_lst5, cor_lst6, cor_lst7, cor_lst8, cor_lst9, num_ev, cor_lst = matching_rate_M2_mixed_CC(ns[i], k_neigh, n_col, num_coords, M, data_generator_path_graph, ns[i])
    et = time.time()
    total_time = et - st
    run_time.append(total_time)
    num_evs.append(num_ev)

    # M
    median_spe_M.append(np.median(np.abs(cor_lst)))
    mean_spe_M.append(np.mean(np.abs(cor_lst)))
    std_spe_M.append(np.std(np.abs(cor_lst)))

    median_spe_M_1.append(np.median(np.abs(cor_lst1)))
    mean_spe_M_1.append(np.mean(np.abs(cor_lst1)))
    std_spe_M_1.append(np.std(np.abs(cor_lst1)))

    median_spe_M_2.append(np.median(np.abs(cor_lst2)))
    mean_spe_M_2.append(np.mean(np.abs(cor_lst2)))
    std_spe_M_2.append(np.std(np.abs(cor_lst2)))

    median_spe_M_3.append(np.median(np.abs(cor_lst3)))
    mean_spe_M_3.append(np.mean(np.abs(cor_lst3)))
    std_spe_M_3.append(np.std(np.abs(cor_lst3)))

  return num_nodes, run_time, median_spe_M_1, median_spe_M_2, median_spe_M_3, mean_spe_M_1, mean_spe_M_2, mean_spe_M_3, std_spe_M_1, std_spe_M_2, std_spe_M_3, num_evs, median_spe_M, mean_spe_M, std_spe_M

et = time.time()
total_time = et - st
print("Execution time: ", total_time, " seconds")

m = 25
num_nodes, run_time, median_spe_M_1, median_spe_M_2, median_spe_M_3, mean_spe_M_1, mean_spe_M_2, mean_spe_M_3, std_spe_M_1, std_spe_M_2, std_spe_M_3, num_evs, median_spe_M, mean_spe_M, std_spe_M = GM_sys_M2_mixed_CC(n_lst, m, num_coords, n_col)

In [13]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) # Force remount to refresh credentials


Mounted at /content/drive


In [1]:
!git clone https://github.com/Nofar-Gabay-30/Unsupervised-Data-Fusion-Via-Graph-Matching.git


Cloning into 'Unsupervised-Data-Fusion-Via-Graph-Matching'...


In [15]:
!cp "/content/drive/MyDrive/Unsupervised Data Fusion Via Graph Matching - Path Graph.ipynb" "/content/Unsupervised Data Fusion Via Graph Matching/"


cp: cannot stat '/content/drive/MyDrive/Unsupervised Data Fusion Via Graph Matching - Path Graph.ipynb': No such file or directory
